# Apriori Example 1

This is a notebook that works through the Apriori Algorithm for example 1.

Primitive functions are defined here, so that the algorithm can be stepped though. This is not computationally efficient, but it's good for illustrating better the algorithmic steps.

The algorithm flow is the following:

- For each level `N` generate all pairwise combinations of sets from level `N-1` that are size `N` (`generate_combinations`)
    - For Level 1, take the set of all single items as one member sets
- For all the combinations, test if there is sufficient *support* in the target combinations (`test_membership` + `filter_membership`)
- If any of the sets at `N-1` did not contribute to a surviving supported set at `N` it is an endpoint *final set* (`final_sets`)

When the surviving combinations is an empty set stop. The union of all of the final sets is the final output.

In [ ]:
"""
    generate_combinations(source_sets, n)

Generate all unique combinations of sets from `source_sets` that have exactly
`n` elements when combined.
"""
function generate_combinations(source_sets, n)
    combinations = Set{Set}()
    for set1 in source_sets
        for set2 in source_sets
            if set1 != set2
                combined = set1 ∪ set2
                if length(combined) == n
                    push!(combinations, combined)
                end
            end
        end
    end
    combinations
end

generate_combinations

In [ ]:
"""
    test_membership(test_sets, target_sets)

Test the membership of a set of sets against another set of sets, returning the
count of how many sets in the second set contain each set from the first set.
"""
function test_membership(test_sets, target_sets)
    occurrences = Dict{Set, Int}()

    for e in test_sets
        count = 0
        for x in target_sets
            if e ⊆ x
                count += 1
            end
        end
        occurrences[e] = count
    end
    occurrences
end

test_membership

In [ ]:
"""
    filter_membership(occurrences, threshold)

Filter the occurrences dictionary of {sets, occurrences} to return only those
sets that have a count greater than or equal to the threshold.

The return value is a set of sets that meet the threshold criteria.
"""
function filter_membership(occurrences, threshold)
    passed = Set()
    for (k, v) in occurrences
        if v >= threshold
            push!(passed, k)
        end
    end
    return passed
end

filter_membership

In [ ]:
"""
    final_sets(source_sets, combined_sets)

Return the final sets from `source_sets` that are not contained in any of the
sets in `combined_sets`.

This is used to find "final" sets that did not generate any supported subsets at
the next level.
"""
function final_sets(source_sets, combined_sets)
    final = Set{Set}()
    for set1 in source_sets
        contained = false
        for set2 in combined_sets
            if set1 ⊆ set2
                contained = true
                break
            end
        end
        if !contained
            push!(final, set1)
        end
    end
    return final
end

final_sets

We use integers to represent the elements $i_n$, here for $n \in 1, 2, \dots 6$. Set $S$ is generated.

In [5]:
S = Set()
for i ∈ 1:6
    push!(S, Set([i]))
end

$X$ is the set of all observation sets that we are interested in.

In [6]:
X = Set([Set([1, 2, 3, 4]), Set([1, 2, 3, 5]), Set([1, 2, 5, 6]), Set([2, 3, 5, 6]), Set([3, 4, 5, 6])])

Set{Set{Int64}} with 5 elements:
  Set([5, 2, 3, 1])
  Set([4, 2, 3, 1])
  Set([5, 6, 2, 3])
  Set([5, 4, 6, 3])
  Set([5, 6, 2, 1])

This is the required level of support for subsets as an integer count. $\epsilon$ is this number divided by the number of sets in $X$. i.e., $\epsilon = 2/5$.

In [7]:
support = 2

2

Level 1 sets are just a copy of $S$...

In [8]:
L1=copy(S)

Set{Any} with 6 elements:
  Set([3])
  Set([5])
  Set([1])
  Set([6])
  Set([2])
  Set([4])

Generate L2 candidates, from all L1 pair combinations

In [9]:
L2_candidates = generate_combinations(L1, 2)

Set{Set} with 15 elements:
  Set([5, 1])
  Set([2, 3])
  Set([6, 2])
  Set([5, 6])
  Set([4, 2])
  Set([4, 1])
  Set([2, 1])
  Set([4, 3])
  Set([6, 1])
  Set([5, 3])
  Set([3, 1])
  Set([6, 3])
  Set([4, 6])
  Set([5, 4])
  Set([5, 2])

Filter the candidates against the support level we require

In [10]:
L2 = filter_membership(test_membership(L2_candidates, X), support)

Set{Any} with 10 elements:
  Set([5, 3])
  Set([4, 3])
  Set([5, 1])
  Set([3, 1])
  Set([6, 3])
  Set([2, 3])
  Set([6, 2])
  Set([5, 6])
  Set([2, 1])
  Set([5, 2])

Same for L3, take the L2 combinations and then filter by those with sufficient support

In [11]:
L3_candidates = generate_combinations(L2, 3)

Set{Set} with 14 elements:
  Set([6, 2, 1])
  Set([5, 2, 3])
  Set([6, 2, 3])
  Set([4, 6, 3])
  Set([6, 3, 1])
  Set([2, 3, 1])
  Set([5, 6, 1])
  Set([4, 2, 3])
  Set([5, 6, 3])
  Set([4, 3, 1])
  Set([5, 3, 1])
  Set([5, 2, 1])
  Set([5, 6, 2])
  Set([5, 4, 3])

In [12]:
L3 = filter_membership(test_membership(L3_candidates, X), support)

Set{Any} with 5 elements:
  Set([5, 6, 2])
  Set([5, 2, 3])
  Set([2, 3, 1])
  Set([5, 2, 1])
  Set([5, 6, 3])

In [13]:
L4_candidates = generate_combinations(L3, 4)

Set{Set} with 3 elements:
  Set([5, 2, 3, 1])
  Set([5, 6, 2, 3])
  Set([5, 6, 2, 1])

In [14]:
L4 = filter_membership(test_membership(L4_candidates, X), support)

Set{Any}()

Now we have nothing left to try, so we can go back and pop the number of final sets at steps 2 and 3 for the answer

In [17]:
answer = final_sets(L2, L3) ∪ final_sets(L3, L4)

Set{Set} with 6 elements:
  Set([5, 6, 2])
  Set([4, 3])
  Set([5, 2, 3])
  Set([2, 3, 1])
  Set([5, 2, 1])
  Set([5, 6, 3])